In [1]:
# NOTEBOOK 5: PROTOTIPO SISTEMA DI RACCOMANDAZIONE
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import accuracy_score
import os

print("PROTOTIPO SISTEMA DI RACCOMANDAZIONE MUSICALE")
print("=" * 55)

# 1. CARICAMENTO COMPONENTI DEL SISTEMA
print("Caricamento componenti del sistema...")

# Caricamento dataset completo
df = pd.read_csv('../data/processed/audio_emotion_dataset.csv')
print(f"Dataset caricato: {len(df)} canzoni")

# Caricamento modello addestrato, scaler ed encoder
try:
    model = joblib.load('../models/best_emotion_classifier.pkl')
    scaler = joblib.load('../models/scaler.pkl')
    le = joblib.load('../models/label_encoder.pkl')
    print("Modello, scaler e encoder caricati")
except FileNotFoundError:
    print("File non trovati. Esegui prima il Notebook 4.")
    exit()

# 2. FUNZIONE PRINCIPALE DI RACCOMANDAZIONE
def recommend_songs(emotion, n_recommendations=5, return_probabilities=False):
    """
    Raccomanda canzoni basandosi sull'emozione predetta dal modello
    
    Args:
        emotion (str): Emozione desiderata ('Happy', 'Sad', 'Angry', 'Relaxed')
        n_recommendations (int): Numero di canzoni da raccomandare
        return_probabilities (bool): Se restituire anche le probabilità
    
    Returns:
        DataFrame con canzoni raccomandate
    """
    
    # Verifica che l'emozione sia valida
    if emotion not in le.classes_:
        available_emotions = list(le.classes_)
        return f"Emozione non valida. Scegli tra: {available_emotions}"
    
    # Preparazione features per la predizione (no colonne metadati)
    features = df.drop(columns=['Track_ID', 'Title', 'Valence', 'Arousal', 'Emotion_Label'])
    
    # Standardizzazione features (COME IN FASE DI TRAINING)
    features_scaled = scaler.transform(features)
    
    # Predizione emozioni per TUTTE le canzoni
    if return_probabilities:
        # Se il modello supporta le probabilità
        try:
            predictions = model.predict_proba(features_scaled)
            emotion_index = list(le.classes_).index(emotion)
            confidence_scores = predictions[:, emotion_index]
        except:
            confidence_scores = np.ones(len(df))  # Fallback
    else:
        predictions = model.predict(features_scaled)
        confidence_scores = (predictions == le.transform([emotion])[0]).astype(int)
    
    # Aggiunta delle predizioni al dataframe
    df_recommend = df.copy()
    df_recommend['Predicted_Emotion'] = le.inverse_transform(predictions)
    df_recommend['Confidence'] = confidence_scores
    
    # Filtro canzoni predette con l'emozione richiesta
    recommended_songs = df_recommend[df_recommend['Predicted_Emotion'] == emotion]
    
    # Ordinamento per confidence (più alta prima)
    recommended_songs = recommended_songs.sort_values('Confidence', ascending=False)
    
    # Return prime n raccomandazioni
    return recommended_songs[['Track_ID', 'Title', 'Emotion_Label', 'Predicted_Emotion', 'Confidence']].head(n_recommendations)

# 3. TEST DEL SISTEMA
print("\nTEST DEL SISTEMA DI RACCOMANDAZIONE")
print("-" * 40)

# Test con diverse emozioni
test_emotions = ['Happy', 'Sad', 'Angry', 'Relaxed']

for emotion in test_emotions:
    print(f"\nRACCOMANDAZIONI PER '{emotion}':")
    recommendations = recommend_songs(emotion, n_recommendations=3)
    
    if isinstance(recommendations, str):
        print(recommendations)
    else:
        for idx, row in recommendations.iterrows():
            status = "ok" if row['Emotion_Label'] == row['Predicted_Emotion'] else "warning"
            print(f"   {status} {row['Title']} (Originale: {row['Emotion_Label']}, Confidence: {row['Confidence']:.3f})")

# 4. VALUTAZIONE DEL PROTOTIPO
print("\nVALUTAZIONE DEL SISTEMA COMPLETO")
print("-" * 35)

# Preparazione dati per la valutazione
X = df.drop(columns=['Track_ID', 'Title', 'Valence', 'Arousal', 'Emotion_Label'])
y_true = df['Emotion_Label']

# Standardizzazione e predizione
X_scaled = scaler.transform(X)
y_pred = le.inverse_transform(model.predict(X_scaled))

# Calcolo accuracy sul dataset completo
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuratezza sul dataset completo: {accuracy:.3f}")

# Accuracy per ogni emozione
from sklearn.metrics import classification_report
print("\nREPORT DETTAGLIATO:")
print(classification_report(y_true, y_pred, target_names=le.classes_))

# 5. INTERFACCIA SEMPLICE
print("\nINTERFACCIA INTERATTIVA SEMPLICE")
print("-" * 35)
print("Prova il sistema! Scrivi un'emozione o 'exit' per uscire.")

def simple_interface():
    while True:
        user_input = input("\nInserisci un'emozione (Happy/Sad/Angry/Relaxed): ").strip().capitalize()
        
        if user_input.lower() == 'exit':
            print("Arrivederci!")
            break
        
        if user_input not in le.classes_:
            print(f"Emozione non valida. Scegli tra: {list(le.classes_)}")
            continue
        
        print(f"\nEcco 5 canzoni consigliate per '{user_input}':")
        recommendations = recommend_songs(user_input, n_recommendations=5)
        
        for idx, row in recommendations.iterrows():
            print(f"   • {row['Title']} (ID: {row['Track_ID']})")

# Per eseguire l'interfaccia (commentata per non bloccare l'esecuzione)
# simple_interface()

print("\nPROTOTIPO COMPLETATO!")
print("=" * 55)

PROTOTIPO SISTEMA DI RACCOMANDAZIONE MUSICALE
Caricamento componenti del sistema...
✅ Dataset caricato: 1546 canzoni
✅ Modello, scaler e encoder caricati

TEST DEL SISTEMA DI RACCOMANDAZIONE
----------------------------------------

RACCOMANDAZIONI PER 'Happy':
   warning Song_1801 (Originale: Relaxed, Confidence: 1.000)
   warning Song_2 (Originale: Relaxed, Confidence: 1.000)
   warning Song_3 (Originale: Relaxed, Confidence: 1.000)

RACCOMANDAZIONI PER 'Sad':
   ok Song_20 (Originale: Sad, Confidence: 1.000)
   warning Song_35 (Originale: Relaxed, Confidence: 1.000)
   warning Song_54 (Originale: Relaxed, Confidence: 1.000)

RACCOMANDAZIONI PER 'Angry':
   ok Song_1798 (Originale: Angry, Confidence: 1.000)
   warning Song_4 (Originale: Happy, Confidence: 1.000)
   warning Song_8 (Originale: Relaxed, Confidence: 1.000)

RACCOMANDAZIONI PER 'Relaxed':
   ok Song_1802 (Originale: Relaxed, Confidence: 1.000)
   ok Song_12 (Originale: Relaxed, Confidence: 1.000)
   ok Song_44 (Originale: